# La régression linéaire

Mettons que vous disposez des mesures annuelles de précipitations en millimètres sur la commune de Pont-Aven dans le Finistère depuis les années 2000 :

In [ ]:
rainfall = [974.5, 879.3, 997.2, 1128.5, 864.9, 864.9, 1136.9, 1015.1, 829.6, 981.4, 830.5, 830.1, 936.3, 613.0, 641.3, 815.7, 748.1, 947.1, 763.8, 688.2, 1119.8, 866.1, 910.1, 686.3, 876.9]

En calculant la moyenne, vous obtenez une évaluation des précipitations sur un an :

In [ ]:
mean = sum(rainfall) / len(rainfall)

Vous pourriez en conclure que, dans un contexte idéal sans variation notable ni accélération de la dérive climatique, `mean` est un estimateur correct des précipitations pour l’année suivante mais avec une certaine marge d’erreur. Quelle est cette erreur ?

La manière la plus intuitive de la déterminer est de prendre l’écart de chaque valeur avec `mean` puis d’en faire une moyenne. Et comme deux écarts positif et négatif se compensent tant et si bien que l’on pourrait faussement conclure que `mean` prédit sans erreur les précipitations pour l’année suivante et toutes celles après elle, il est de bon ton soit de considérer les valeurs absolues soit de les élever au carré. Avec cette deuxième option, nous calculons la variance empirique du système :

$$
s^2 = \frac{1}{n} \sum_{i=1}^{n} (x_i - \bar{x})^2
$$

In [ ]:
variance = sum((x - mean) ** 2 for x in rainfall) / len(rainfall)

Pour tenir compte du biais de l’échantillonnage, on a plutôt coutume de calculer la variance empirique corrigée (ou non biaisée) :

$$
\sigma^2 = \frac{1}{n-1} \sum_{i=1}^{n} (x_i - \bar{x})^2
$$

In [ ]:
variance = sum((x - mean) ** 2 for x in rainfall) / (len(rainfall) - 1)

Cette mesure s’exprimant dans une unité qui ne fait pas immédiatement sens, à savoir les $mm^2$, on les rapporte à une échelle compréhensible en prenant la racine carrée, de manière à obtenir l’écart-type non biaisé :

$$
\sigma = \sqrt{\frac{1}{n-1} \sum_{i=1}^{n} (x_i - \bar{x})^2}
$$

In [ ]:
deviation = variance ** .5

print(f"Les précipitations moyennes au-dessus de la commune de Pont-Aven avoisinent les {round(mean)} mm avec un écart-type de {round(deviation)} mm.")

Le modèle que nous venons de construire repose essentiellement sur une mesure de tendance centrale calculée à partir d’une seule variable, la pluviométrie. Si nous lui rajoutions une seconde variable, par exemple la température annuelle moyenne, nous disposerions alors de deux coordonnées qu’il serait intéressant d’analyser soit visuellement en les projetant dans un plan en deux dimensions soit en mesurant leurs interactions.

L‘une des façons de faire serait par exemple de calculer une fonction affine, autrement appelée droite de régression linéaire, qui s’exprime sous la forme :

$$
y = mx + b
$$

Pour la petite histoire, c’est en 1886 que le terme de régression apparaît pour la première fois, dans un article de Sir Francis Galton qu’il publie sous le titre de *Regression Towards Mediocrity in Hereditary Stature*. Dans cet article, il mettait en évidence que les enfants de personnes de grandes tailles avaient tendance à être plus petits qu’elles, et inversement, d’où l’idée d’une régression vers la médiocrité pour décrire un phénomène d’attraction vers la moyenne.

Avant de monter notre modèle, chargeons toutes les bibliothèques nécessaires :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.stats import pearsonr, shapiro, levene, bartlett

## Rappels sur l’équation réduite d’une droite

Dans un plan orthonormé, une droite passe au minimum par deux points de coordonnées $(x1;y1)$ et $(x2;y2)$. Elle est caractérisée par une pente, son coefficient directeur, et par son ordonnée à l’origine, c’est-à-dire l’ordonnée de son point d’intersection avec l’axe des ordonnées. Dans la formule $y = mx + b$, le terme $m$ est le coefficient directeur et $b$ l’ordonnée à l’origine.

Commençons par matérialiser une droite quelconque dans un plan en considérant les points de coordonnées $(2;3)$ et $(4;9)$ :

In [ ]:
x = [2, 4]
y = [3, 9]

plt.xlim([0, 10])
plt.ylim([0, 10])
plt.grid(True)
sns.regplot(x=x, y=y, ci=None)
sns.despine()
plt.show()

### Le coefficient directeur

Le coefficient directeur détermine la pente de la droite : s’il est positif, la droite monte ; s’il est négatif, la droite descend. Pour le calculer, la formule est :

$$m = \frac{\Delta y}{\Delta x}$$

In [ ]:
m = (y[0] - y[1]) / (x[0] - x[1])

print(m)

### L’ordonnée à l’origine

Maintenant que nous connaissons *m*, il reste à calculer *b*, qui est l’ordonnée à l’origine. Or, nous savons que l’un des points de cette droite a pour coordonnées $(2;3)$, qui en est une solution. C’est-à-dire que lorsque *x* vaut 2, alors *y* vaut 3. Le droite qui nous occupe a donc pour équation :

$$y = 3x + b$$

D’où :

$$
\begin{align*}
3 &= 3 \times 2 + b \\
b &= -3 \times 2 + 3 \\
b &= -3
\end{align*}
$$

In [ ]:
b = -3 * 2 + 3

print(b)

Nous pouvons afficher cet autre point de coordonnées $(0;-3)$ sur la droite afin de vérifier graphiquement si la solution est correcte :

In [ ]:
x = [2, 4, 0]
y = [3, 9, -3]

plt.xlim([-1, 10])
plt.ylim([-4, 10])
plt.grid(True)
sns.regplot(x=x, y=y, ci=None)
sns.despine()
plt.show()

Nous pouvons ainsi conclure que l’équation réduite de la droite est : $$y = 3x -3$$

## Une droite de régression

L’exemple introductif montrait comment calculer l’équation réduite d’une droite dont on connaît deux points. Le problème dans le cas de la régression, c’est que justement nous ne les connaissons pas ces points.

Avant d’établir un modèle linéaire, il est judicieux d’examiner la relation entre deux variables afin de déterminer si elles se prêtent bien à l’exercice. On peut le faire soit visuellement soit à l’aide d’un coefficient.

### Vérifier graphiquement si deux variables sont corrélées de manière linéaire

Matérialisons concrètement la question avec un jeu de données sur des manchots en Antarctique ([Gorman, 2014](0.about-datasets.ipynb#Size-measurements-for-adult-foraging-penguins-near-Palmer-Station,-Antarctica)) :

In [ ]:
df = pd.read_csv("./data/penguin-census.csv")

fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12,5))
sns.scatterplot(data=df, x="body_mass_g", y="flipper_length_mm", ax=ax1)
sns.regplot(data=df, x="body_mass_g", y="flipper_length_mm", ci=None, ax=ax2)
sns.despine()

Nous venons d’afficher deux nuages de points qui représentent deux caractéristiques des manchots : la masse en grammes sur l’axe des abscisses et la longueur des nageoires en millimètres sur l’axe des ordonnées. Nous notons clairement une tendance : lorsque la masse d’un manchot augmente, la longueur de ses nageoires aussi. Ceci dit, la relation entre les deux caractéristiques n’est pas strictement linéaire : aucun individu n’est identique et nous ne pouvons être sûr·es que, si un manchot pèse *x* grammes de plus qu’un autre, ses nageoires mesureront *y* mm de plus.

Sur le graphique de droite, nous avons en plus affiché la droite de régression. Cette droite nous montre la tendance centrale qui minimise la somme des erreurs pour toutes les observations et qui nous permettrait en plus de prédire pour une certaine masse la longueur des nageoires d’un manchot.

### Déterminer la corrélation à l’aide d’un coefficient

Les coefficients de corrélation et de détermination de Pearson sont deux manières d’analyser la relation entre deux variables. La bibliothèque *Scipy* met à disposition une fonction `pearsonr()` pour les obtenir directement.

#### Le coefficient de corrélation de Pearson (*r*)

Il mesure la force et la direction de la relation linéaire entre deux variables dans un intervalle $[-1,1]$ où un *r* proche de 1 ou -1 indique une forte corrélation linéaire, tandis qu’un *r* proche de 0 indique une absence de relation linéaire. Il s’exprime mathématiquement par la formule :

$$
r = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i=1}^n (x_i - \bar{x})^2 \cdot \sum_{i=1}^n (y_i - \bar{y})^2}}
$$

In [ ]:
r, _ = pearsonr(df.dropna().body_mass_g.values, df.dropna().flipper_length_mm.values)

print(f"r de Pearson : {r}")

#### Le coefficient de détermination linéaire de Pearson ($R^2$)

Le $R^2$ est la somme des carrés des résidus et de la variance totale :

$$
R^2 = 1 - \frac{\sum_{i=1}^k(y_i - \hat{y}_i)^2}{\sum_{i=1}^k(y_i - \bar{y})^2}
$$

Il mesure la qualité de la prédiction d’un modèle de régression linéaire en évaluant la variance d’une variable en fonction d’une autre dans l’intervalle $[0,1]$ où un $R^2$ de 1 est un score parfait quand un score de 0 indiquerait que le modèle prédit toujours la valeur attendue (la moyenne). Un score négatif reste possible mais serait révélateur d’une erreur de méthodologie (données arbitrairement mauvaises).

In [ ]:
print(f"R^2 de Pearson : {r ** 2}")

## La droite de régression des moindres carrés

Lorsque nous prenons les coordonnées d’une observation, nous remarquons bien qu’elles sont éloignées de la droite. Il existe un décalage, que l’on appelle communément une erreur, et la droite de régression des moindres carrés est celle qui minimise la somme des carrés de toutes les erreurs, le rapport s’établissant au carré afin d’éviter les valeurs négatives.

### La formule

L’équation réduite qui permet d’obtenir les coordonnées de tous les points de la droite et, partant, d’obtenir une prédiction de $\hat{y}$ en fonction de $x$, respecte la forme $\hat{y} = mx + b$. Deux étapes majeures pour la trouver, calculer d’abord le coefficient directeur $m$ puis l’ordonnée à l’origine $b$.

### Calculer le coefficient directeur

La résolution du coefficient directeur d’une droite des moindres carrés est régi par la formule ci-dessous, où $n$ est le nombre des observations :

$$m = \frac{n\sum xy - \sum x \sum y}{n \sum x^2 - \left(\sum x\right)^2}$$

Commençons par définir un nouveau *data frame* avec l’ensemble des valeurs pour les caractéristiques *body_mass_g* et *flipper_length_mm* :

In [ ]:
# for educational purposes, NA have been removed
data = df.loc[:,["body_mass_g","flipper_length_mm"]].dropna()

display(data.head())

Récupérons individuellement les coordonnées *x* et *y* qui représentent respectivement la variable prédictive et la variable cible :

In [ ]:
X, Y = data.body_mass_g.values, data.flipper_length_mm.values

Il ne nous reste plus qu’à créer une fonction `slope()` qui implémente la formule plus haut afin de calculer le coefficient directeur :

In [ ]:
def slope(x, y):
  """Return the slope of a straight line,
  with the least squares method.

  Arguments:
  x -- data in x-axis
  y -- data in y-axis
  """

  n = len(x)
  sum_xy = sum(x * y)
  sum_x_squared = sum(x) ** 2
  sum_x = sum(x)
  sum_y = sum(y)
  sum_squares_x = sum(x ** 2)

  # formula
  m = (n * sum_xy - sum_x * sum_y) / (n * sum_squares_x - sum_x_squared)

  return m

m = slope(X, Y)

print(f"Le coefficient directeur de la droite des moindres carrés est approximativement de {m:.4f}.")

### Calculer l’ordonnée à l’origine

La formule de résolution de l’ordonnée à l’origine fait appel au coefficient directeur et aux moyennes des valeurs de $x$ et de $y$ :

$$b = \bar{y} - m\bar{x}$$

Nous pouvons traduire cette formule dans une fonction `intercept()` et la calculer :

In [ ]:
def intercept(m, x, y):
    """Intercept of a straight line.
    
    Arguments:
    m -- weight
    x -- data in x-axis
    y -- data in y-axis
    """

    avg_x = np.average(x)
    avg_y = np.average(y)
    
    return avg_y - (m * avg_x)

b = intercept(m, X, Y)

print(f"L’ordonnée à l’origine de la droite des moindres carrés est approximativement de {b:.4f}.")

De là, nous pouvons conclure que la fonction affine qui modélise la relation linéaire entre la masse en grammes (*x*) et la longueur des nageoires des manchots (*y*) est de :

In [ ]:
print(f"ŷ = {m}x + {b}")

### Tracer la droite

Grâce à la fonction affine obtenue précédemment, il est à présent simple d’obtenir des prédictions en fonction d’une coordonnée ou de l’autre. Définissions une fonction `predict()` pour prédire la longueur des nageoires d’un manchot à partir du moment où l’on connaît son poids :

In [ ]:
def predict(x, m, b):
    """Solution to the standard form equation
    of a straight line.

    Keyword-only arguments:
    x -- value of x
    m -- slope
    b -- intercept
    """
    return m * x + b

La question est désormais de savoir, pour tous les manchots de notre jeu de données, quelle devrait être la longueur de leurs nageoires en situation idéale. Nous pouvons le déterminer grâce à notre fonction :

In [ ]:
Y_pred = [ predict(x, m, b) for x in X ]

Nous pouvons maintenant comparer si notre modèle linéaire est similaire à celui proposé plus haut par *Seaborn* :

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# predictions with our linear model
sns.lineplot(x=X, y=Y_pred, color="fuchsia", ax=axes[0], label="Linear model")
sns.scatterplot(x=X, y=Y, color="seagreen", ax=axes[0], label="Observations")
axes[0].set(xlabel="Body mass (g)", ylabel="Flipper length (mm)")
axes[0].legend()
sns.despine(ax=axes[0])

# linear regression with regplot
sns.regplot(data=df, x="body_mass_g", y="flipper_length_mm", ci=None, ax=axes[1])
axes[1].set(xlabel="Body mass (g)", ylabel="Flipper length (mm)")
sns.despine(ax=axes[1])

plt.tight_layout()
plt.show()

## Établir une mesure de la performance du modèle

Si la droite obtenue par un calcul manuel correspond au graphique attendu, il reste encore à évaluer la performance du modèle. Nous abordons trois métriques possibles.

### L’erreur quadratique moyenne (MSE)

La MSE (*mean square error*) et sa cousine, la RMSE (*root mean square error*), sont les deux métriques les plus couramment utilisées en *machine learning*. La MSE calcule la moyenne des carrés des erreurs selon la formule :

$$
\text{MSE} = \frac{1}{k}\sum_{i=0}^{k-1}(y_i - \hat{y}_i)^2
$$

Comme entre en jeu un calcul au carré, la MSE pénalise plus fortement les grandes erreurs et, dans le même ordre d’idée, sera très sensible aux données aberrantes (*outliers*).

In [ ]:
def mean_squared_error(data, predictions):

    # formula
    mse = sum((y - y_pred) ** 2 for y, y_pred in zip(data, predictions)) / len(data)
    
    return mse

print(f"MSE : {mean_squared_error(Y, Y_pred)}")

### La racine de l’erreur quadratique moyenne (RMSE)

Plus facile à interpréter que la MSE, la RMSE (*root mean square error*) s’exprime dans l’unité de la variable à prédire en extrayant la racine carrée de la MSE :

$$
\text{RMSE} = \sqrt{ \frac{1}{k}\sum_{i=0}^{k-1}(y_i - \hat{y}_i)^2 }
$$

À noter qu’elle souffre des mêmes limites que la MSE : une grande sensibilité aux *outliers* ainsi qu’une incidence forte sur les erreurs importantes.

In [ ]:
def root_mean_squared_error(data, predictions):

    # formula
    mse = sum((y - y_pred) ** 2 for y, y_pred in zip(data, predictions)) / len(data)
    
    return mse ** .5

print(f"RMSE : {root_mean_squared_error(Y, Y_pred)} mm")

### L’erreur absolue moyenne

Quand les valeurs extrêmes d’un jeu de données sont quantitativement importantes, la RMSE pourrait conduire à des erreurs d’interprétation. Dans un tel cas de figure, la MAE (*mean absolute error*) peut lui être préférée : en calculant la moyenne de valeurs absolues, elle ne pénalise plus autant les grandes erreurs et se rend moins sensible aux données aberrantes. La formule de la MAE vaut ainsi :

$$
\text{MAE} = \frac{1}{k}\sum_{i=0}^{k-1} \lvert y_i - \hat{y}_i\rvert
$$

In [ ]:
def mean_absolute_error(data, predictions):

    mae = sum(abs(d - p) for d, p in zip(data, predictions)) / len(data)
    return mae

print(f"MAE : {mean_absolute_error(Y, Y_pred)} mm")

## Analyse des résidus

Nous avons établi plus haut la fonction affine qui permet de tracer la droite de régression qui minimise la moyenne des écarts à la moyenne :

$$
y = mx + b
$$

Nous avons également relevé qu’il existait, entre cette droite idéale tracée par les prédictions et les valeurs observées empiriquement, un écart que nous avons nommé *erreur*. Dans le modèle de régression linéaire, nous ajoutons ce paramètre à la fonction affine en terme de *bruit* :

$$
y = \beta_0 + \beta_1 x + \epsilon
$$

La notion de bruit traduit la variabilité entre les prédictions du modèle et les observations réelles et nous permet d’identifier ses lacunes en vérifiant notamment si les hypothèses fondamentales, comme l’homoscédasticité et la linéarité, sont bien respectées. On effectue pour cela une **analyse des résidus**.

Les résidus, ce sont ces écarts entre les prédictions et la variable indépendante qui constituent la partie inexpliquée de notre modèle :

In [ ]:
residuals = Y - Y_pred

Plus que de simples erreurs sans signification, leur distribution peut nous renseigner sur la précision du modèle : si elle est aléatoire et de faible amplitude, elle révèlera que le modèle capture bien les structures sous-jacentes des données ; à l’inverse, des résidus structurés ou de grande amplitude suggèreront que le modèle est inadéquat.

### Visualisation des résidus

En affichant le **terrain résiduel**, nous devons être en mesure de confirmer les hypothèses fondamentales de tout modèle linéaire, à savoir **la linéarité** et **l’homoscédasticité** (la variance des résidus est constante). Si les points sont répartis aléatoirement autour de la ligne de référence qui matérialise l’absence de résidus, tout va bien ! Si au contraire on remarque des regroupements ou si un motif apparaît, comme une courbe, alors notre modèle n’est pas adapté.

Dans notre exemple, le phénomène aléatoire nous semble bien retranscrit :

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=Y_pred, y=residuals, marker='^', alpha=0.7, s=80, linewidth=0.5)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residual plot', fontsize=14)
plt.xlabel('Predicted values', fontsize=12)
plt.ylabel('Residuals', fontsize=12)
plt.grid(alpha=0.3)
sns.despine()
plt.show()

### Tester la normalité d’une distribution

Afin de prouver que la distribution des résidus est un phénomène aléatoire, on peut la soumettre à un test statistique comme celui de Shapiro-Wilk qui vérifie sa normalité. L’hypothèse nulle $H_0$ établit que les résidus sont normalement distribués.

Avec une valeur-p inférieure au seuil de 0,05, nous la rejetons et devons conclure que notre modèle n’est pas adéquat à décrire les données :

In [ ]:
stat, p_value = shapiro(residuals)
print(
    f"Score : {stat}",
    f"Valeur-p : {p_value}",
    sep="\n"
)

D’autres tests existent, comme celui de Kolmogorov-Smirnov (plus sensible aux écarts dans de grands ensembles d’échantillons) ou encore celui de Anderson-Darling qui va définir des seuils critiques sans calculer une valeur-p directement.

### Tester l’homoscédasticité d’une distribution

Afin de déterminer si la variance est constante dans les résidus, nous pouvons constituer deux groupes séparés par la médiane :

In [ ]:
group1 = residuals[Y_pred < np.median(Y_pred)]
group2 = residuals[Y_pred >= np.median(Y_pred)]

Et lancer le test de Bartlett qui évalue si leurs variances sont égales :

In [ ]:
stat, p_value = bartlett(group1, group2)

print(
    f"Score : {stat}",
    f"Valeur-p : {p_value}",
    sep="\n"
)

Avec une valeur-p supérieure au seuil de 0,05, nous pouvons conserver l’hypothèse nulle d’homoscédasticité.

Malgré tout, gardons à l’esprit que nos résidus ne sont pas normalement distribués et optons pour un test moins sensible à la contrainte de normalité :

In [ ]:
stat, p_value = levene(group1, group2)

print(
    f"Score : {stat}",
    f"Valeur-p : {p_value}",
    sep="\n"
)

La valeur-p est toujours supérieure au seuil 0,05, ce qui tend à valider l’hypothèse nulle de constance de la variance entre les différents groupes.